# OCR on pdf & jpg

In this notebook we want use OCR on pdf and jpg for extracting information and matching this with a dataset

In [ ]:
pip install  pypdf langchain tika typing fuzzywuzzy pymupdf

In [12]:
import pandas as pd
from langchain.document_loaders import PyPDFLoader
from tika import parser
from typing import List
import re
from typing import List, Tuple
from fuzzywuzzy import fuzz, process
import fitz

In [ ]:
#Text ci metti un pdf
def extract_data(text):
    pattern = r'\((\w+)\) - ([A-Z ]+)'
    data = re.findall(pattern, text)
    for i in range(len(data)):
        num = data[i][0]
        num = re.sub(r'^0+', '', num)  # rimuovi gli zeri iniziali
        num = re.sub(r'\D', '', num)  # rimuovi le lettere
        data[i] = (num, data[i][1])
    return data

In [13]:
def match_recipe_to_dataset(pdf_file: str, dataset: pd.DataFrame, threshold: int = 90) -> List[Tuple]:
    if not isinstance(pdf_file, str):
        raise TypeError("Il percorso del file PDF deve essere una stringa.")
    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("Il dataset deve essere un oggetto pandas.DataFrame.")

    # estraiamo il testo dal file pdf
    pdf_file = fitz.open(pdf_file)
    text = ''
    for page in pdf_file:
        text += page.get_text()

    # individuiamo le parole di interesse
    words = re.findall(r' - ([A-Za-z\s]+)', text) #Funzione regex per estrarre parole dalla ricetta (iniziale =' - ([A-Z]+))'

    # matchiamo le parole con le parole della colonna 'COD_PREST_AMBLE' con fuzzywuzzy
    matches = []
    for word in words:
        if not isinstance(word, str):
            raise TypeError("La parola di interesse deve essere una stringa.") #funzione per errori
        scores = process.extract(word, dataset['COD_PREST_AMBLE'], scorer=fuzz.token_set_ratio, limit=1) #qui faccio fuzzymatch devo capire il parametro limit
        filtered_scores = [score for score in scores if score[1] >= threshold]
        for score in filtered_scores:
            rows = dataset[dataset['COD_PREST_AMBLE'] == score[0]].head(1).iterrows() #Decidiamo l'output 
            matches.extend([(word, row[1], score[1]) for row in rows])

    # ordiniamo i risultati in base allo score
    matches_sorted = sorted(matches, key=lambda x: x[2], reverse=True)

    return matches_sorted

NameError: name 'Tuple' is not defined